## Установка, импорт библиотек

In [1]:
import pandas as pd
from corus import load_lenta
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import pymorphy2
import re
from pandarallel import pandarallel
from cleantext import clean
import numpy as np
import random
import os

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.pipeline import Pipeline
import optuna
from optuna.samplers import TPESampler

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


## Фиксируем random state

In [2]:
RANDOM_STATE = 42

def set_seed(seed: int = RANDOM_STATE) -> None:
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(42)

Random seed set as 42


## Загрузка данных новостей

In [3]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [4]:
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [5]:
data = []
for record in records:
    data.append((record.text, record.title, record.topic))

df = pd.DataFrame(data, columns=["text", "title", "topic"])

In [6]:
df

,text,title,topic
0,Австрийские правоохранительные органы не предс...,Австрия не представила доказательств вины росс...,Спорт
1,Сотрудники социальной сети Instagram проанализ...,Обнаружено самое счастливое место на планете,Путешествия
2,С начала расследования российского вмешательст...,В США раскрыли сумму расходов на расследование...,Мир
3,Хакерская группировка Anonymous опубликовала н...,Хакеры рассказали о планах Великобритании зами...,Мир
4,Архиепископ канонической Украинской православн...,Архиепископ канонической УПЦ отказался прийти ...,Бывший СССР
...,...,...,...
739345,Сегодня областной центр Сахалина и Курил получ...,Южно-Сахалинск объявлен очагом холеры,Россия
739346,Бывший шеф Службы безопасности президента Але...,Леворадикалы создают предвыборный блок,Россия
739347,Сегодня утром в районах дагестанских селений Ч...,В горах Дагестана идут активные боевые действия,Россия
739348,Намеченная на сегодняшний день церемония вступ...,Карачаево-Черкесия раскололась по национальном...,Россия


## Предобработка данных

In [7]:
mo = df['text'].apply(len).describe()['mean']
sko = df['text'].apply(len).describe()['std']

Удалим выбросы из нашего датасета

In [8]:
df = df[df.text.apply(len) < mo+3*sko]
df.shape

(732724, 3)

In [9]:
df['topic'].value_counts()

topic
Россия               158851
Мир                  135973
Экономика             78836
Спорт                 64213
Культура              53501
Наука и техника       52627
Бывший СССР           51896
Интернет и СМИ        44220
Из жизни              27575
Дом                   21524
Силовые структуры     19489
Ценности               7755
Бизнес                 7205
Путешествия            6405
69-я параллель         1265
Крым                    664
Культпросвет            340
                        200
Легпром                 113
Библиотека               65
Оружие                    3
ЧМ-2014                   2
МедНовости                1
Сочи                      1
Name: count, dtype: int64

Удалим топики, где сликшом мало объектов, также удалим странный пустой топик

In [10]:
cnt = df['topic'].value_counts()

to_del_topics = list(cnt[cnt < 300].index)
print("Удаленные топики: ", to_del_topics)

Удаленные топики:  ['', 'Легпром', 'Библиотека', 'Оружие', 'ЧМ-2014', 'МедНовости', 'Сочи']


In [11]:
df = df[~df["topic"].isin(to_del_topics)]
df['topic'].value_counts()

topic
Россия               158851
Мир                  135973
Экономика             78836
Спорт                 64213
Культура              53501
Наука и техника       52627
Бывший СССР           51896
Интернет и СМИ        44220
Из жизни              27575
Дом                   21524
Силовые структуры     19489
Ценности               7755
Бизнес                 7205
Путешествия            6405
69-я параллель         1265
Крым                    664
Культпросвет            340
Name: count, dtype: int64

## Получим репрезентативную выборку меньшего размера, возьмем в каждой группе по 15%, что по итогу нам даст около 100K объектов в сэмпле

In [12]:
sample_data = df.groupby('topic', group_keys=False).apply(lambda x: x.sample(frac=0.15))

C:\Users\lsd24\AppData\Local\Temp\ipykernel_22288\3103091495.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_data = df.groupby('topic', group_keys=False).apply(lambda x: x.sample(frac=0.15))


In [13]:
sample_data.reset_index(inplace=True)
sample_data

,index,text,title,topic
0,149397,Ученые из новосибирского Института химической ...,Сибирские ученые открыли новый способ выявлени...,69-я параллель
1,142717,«Норильский никель» планирует в 2016-2018 года...,«Норникель» потратит на инвестиции 6 миллиардо...,69-я параллель
2,50131,В Министерстве труда предложили дарить матерям...,Минтруд собрался раздать новорожденным чепчики...,69-я параллель
3,139634,На полуострове Ямал для туристов разработали м...,Туристам на Ямале предложили посетить самую во...,69-я параллель
4,105236,145 миллиардов рублей будет инвестировано в ст...,В создание Мурманского транспортного узла влож...,69-я параллель
...,...,...,...,...
109846,215891,Airbus Group NV выиграла конкурс China Aircraf...,Airbus поставит Китаю еще 100 самолетов,Экономика
109847,580811,Госдума приняла в третьем чтении поправки в На...,Госдума разрешила ФНС забирать банковские вклады,Экономика
109848,258185,Россия планирует к 2030 году занять 20 процент...,Россия собралась занять 20 процентов мирового ...,Экономика
109849,681012,Крупнейшие инвестиционные компании мира J.P. M...,Инвестиционные компании США готовят масштабные...,Экономика


## Подготовка данных к обучению

Сделаем нормализацию и очистку данных, чтобы улучшить качество данных.

In [14]:
def clean_russian_news(text):
    cleaned_text = clean(
        text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=False,
        no_punct=True,
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_punct="",
        lang='ru'
    )
    return cleaned_text

In [15]:
for t in ("title", "text"):
    sample_data[t] = sample_data[t].apply(clean_russian_news)

Примерный вид текста после обработки:

In [16]:
sample_data['text'][0]

'ученые из новосибирского института химической биологии и фундаментальной медицины ихбфм со ран и красноярского института биофизики ибф со ран создали биосенсор способный с высокой точностью распознавать клещевой энцефалит об этом сообщает со ссылкой на представителей ихбфм со ран разработка представляет собой биолюминисцентный зонд на основе клонированного белка мягкого коралла преимущество биосенсора высокая чувствительность которая позволяет выявить даже незначительное количество вируса клещевого энцефалита говорится в официальном издании со ран наука в сибири для тестирования на энцефалит клещ размалывается из него экстрагируется водная часть которая может содержать вирус затем водная суспензия вносится в ячейки содержащие антитела далее добавляется биолюминисцентный зонд если вирус присутствует зонд подаст сигнал пояснили ученые выявить клещевой энцефалит в крови сложнее он присутствует там недолго а потом проникает в нервную систему так что основное применение зонда экспресспрове

### Лемматизация и удаление стоп слов
Проведем лемматизацию, хоть она и более долгая, но тк стэмминг мне кажется более слабоым видом обработки текста в силу специфики русского языка,

In [17]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lsd24\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lsd24\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lsd24\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\lsd24\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [18]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    Doc
)
from razdel import tokenize  # Для токенизации текста
import nltk
from nltk.corpus import stopwords

# Скачивание стоп-слов из NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# Инициализация компонентов Natasha
segmenter = Segmenter()  # Токенизатор и сегментатор текста
morph_vocab = MorphVocab()  # Морфологический словарь
emb = NewsEmbedding()  # Предобученные эмбединги для анализа
morph_tagger = NewsMorphTagger(emb)  # Морфологический теггер
syntax_parser = NewsSyntaxParser(emb)  # Синтаксический парсер (если нужен)

def txt_processing(text, stop_words=stop_words):
    # Создание объекта Doc для анализа текста
    doc = Doc(text)

    # Сегментация текста на предложения и токены
    doc.segment(segmenter)

    # Морфологический анализ
    doc.tag_morph(morph_tagger)

    # Лемматизация каждого токена
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    # Фильтрация токенов: удаление стоп-слов и неалфавитных символов
    filtered_tokens = [
        token.lemma for token in doc.tokens
        if token.lemma.lower() not in stop_words and token.lemma.isalpha()
    ]

    # Объединение лемм в строку
    processed_text = " ".join(filtered_tokens)

    return processed_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lsd24\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
for t in ("title", "text"):
    sample_data[t] = sample_data[t].apply(txt_processing)

Пример статьи после предобработки

In [20]:
sample_data['text'][0]

'ученый новосибирский институт химический биология фундаментальный медицина ихбфма рана красноярский институт биофизика ибф рана создать биосенсор способный высокий точность распознавать клещевой энцефалит это сообщать ссылка представитель ихбфма рана разработка представлять биолюминисцентный зонд основа клонировать белок мягкий коралл преимущество биосенсор высокий чувствительность который позволять выявить незначительный количество вирус клещевой энцефалит говориться официальный издание рана наука сибирь тестирование энцефалит клещ размалываться экстрагироваться водный часть который мочь содержать вирус затем водный суспензия вноситься ячейка содержать антитело далее добавляться биолюминисцентный зонд вирус присутствовать зонд подать сигнал пояснить ученый выявить клещевой энцефалит кровь сложнее присутствовать недолго проникать нервный система основный применение зонд экспресспроверкий клещ переносить заболевание начало массовый практический применение зонд нужный провести ряд экспе

## Разделим выборку на train test val

In [21]:
train_df, temp_df = train_test_split(sample_data, test_size=0.4, stratify=sample_data["topic"], random_state=RANDOM_STATE)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["topic"], random_state=RANDOM_STATE)

Посмотрим на распределение данных в датастет(в силу стратификации, во всех 3х частях примерно одинаковое распределение)

In [22]:
print(train_df["topic"].value_counts(normalize=True))

topic
Россия               0.216917
Мир                  0.185662
Экономика            0.107647
Спорт                0.087680
Культура             0.073054
Наука и техника      0.071856
Бывший СССР          0.070854
Интернет и СМИ       0.060385
Из жизни             0.037642
Дом                  0.029389
Силовые структуры    0.026612
Ценности             0.010590
Бизнес               0.009847
Путешествия          0.008754
69-я параллель       0.001730
Крым                 0.000910
Культпросвет         0.000470
Name: proportion, dtype: float64


# Используем dummy классификатор со стратифицированным распределением

In [23]:
dummy_clf = DummyClassifier(strategy="stratified", random_state=RANDOM_STATE)
dummy_clf.fit(train_df['text'], train_df['topic'])

y_pred = dummy_clf.predict(val_df['text'])

print(classification_report(val_df['topic'], y_pred))

                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00        38
           Бизнес       0.03      0.02      0.02       216
      Бывший СССР       0.07      0.07      0.07      1557
              Дом       0.02      0.02      0.02       646
         Из жизни       0.05      0.05      0.05       828
   Интернет и СМИ       0.06      0.06      0.06      1326
             Крым       0.00      0.00      0.00        20
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.07      0.07      0.07      1605
              Мир       0.19      0.19      0.19      4079
  Наука и техника       0.07      0.08      0.07      1579
      Путешествия       0.01      0.01      0.01       192
           Россия       0.22      0.22      0.22      4765
Силовые структуры       0.01      0.01      0.01       585
            Спорт       0.09      0.09      0.09      1926
         Ценности       0.00      0.00      0.00       

Как видим, бейзлайн показывает достаточно низкое качество, в частности accuracy = 0.12

## Обучим модель sklearn.linear_model.LogisticRegression с двумя вариантами векторизации

In [24]:
logreg = LogisticRegression(random_state=RANDOM_STATE)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_df['text'])

logreg.fit(X, train_df["topic"])
y_pred = logreg.predict(vectorizer.transform(val_df["text"]))

print(classification_report(val_df['topic'], y_pred))

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                   precision    recall  f1-score   support

   69-я параллель       0.71      0.26      0.38        38
           Бизнес       0.56      0.42      0.48       216
      Бывший СССР       0.82      0.78      0.80      1557
              Дом       0.86      0.76      0.81       646
         Из жизни       0.63      0.58      0.61       828
   Интернет и СМИ       0.73      0.69      0.71      1326
             Крым       0.67      0.30      0.41        20
    Культпросвет        0.50      0.10      0.17        10
         Культура       0.85      0.87      0.86      1605
              Мир       0.78      0.80      0.79      4079
  Наука и техника       0.83      0.82      0.82      1579
      Путешествия       0.75      0.62      0.68       192
           Россия       0.76      0.82      0.79      4765
Силовые структуры       0.61      0.51      0.56       585
            Спорт       0.95      0.96      0.96      1926
         Ценности       0.88      0.85      0.86       

In [25]:
logreg = LogisticRegression(random_state=RANDOM_STATE)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_df['text'])

logreg.fit(X, train_df["topic"])
y_pred = logreg.predict(vectorizer.transform(val_df["text"]))

print(classification_report(val_df['topic'], y_pred))

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00        38
           Бизнес       0.83      0.14      0.24       216
      Бывший СССР       0.82      0.78      0.80      1557
              Дом       0.86      0.72      0.78       646
         Из жизни       0.66      0.56      0.60       828
   Интернет и СМИ       0.77      0.67      0.72      1326
             Крым       0.00      0.00      0.00        20
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.82      0.88      0.85      1605
              Мир       0.79      0.84      0.82      4079
  Наука и техника       0.83      0.86      0.84      1579
      Путешествия       0.76      0.47      0.58       192
           Россия       0.76      0.85      0.80      4765
Силовые структуры       0.72      0.35      0.47       585
            Спорт       0.96      0.97      0.96      1926
         Ценности       0.91      0.74      0.82       

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


При том, что по теории tf-idf векторизатор должен быть лучше, на практие для нашей задачи качество 2х моделей близкое

## Запустим optuna для подбора лучших гиперпараметров на кроссвалидации и проверим качество лучшей модели

In [26]:
def create_vectorizer(trial, vec_type):
    params = {}
    prefix = f"{vec_type}__"
    
    if vec_type == "tfidf":
        params.update({
            "ngram_range": trial.suggest_categorical(f"{prefix}ngram_range", [(1,1), (1,2)]),
            "max_df": trial.suggest_float(f"{prefix}max_df", 0.75, 1.0),
            "min_df": trial.suggest_int(f"{prefix}min_df", 1, 5),
            "use_idf": trial.suggest_categorical(f"{prefix}use_idf", [True, False])
        })
        return TfidfVectorizer(**params)
        
    elif vec_type == "count":
        params.update({
            "ngram_range": trial.suggest_categorical(f"{prefix}ngram_range", [(1,1), (1,2)]),
            "max_df": trial.suggest_float(f"{prefix}max_df", 0.75, 1.0),
            "min_df": trial.suggest_int(f"{prefix}min_df", 1, 5)
        })
        return CountVectorizer(**params)

def build_model_pipeline(trial):
    vectorizer_type = trial.suggest_categorical("vectorizer", ["tfidf", "count"])
    vectorizer = create_vectorizer(trial, vectorizer_type)
    
    logreg_params = {
        "C": trial.suggest_float("logreg__C", 0.1, 10.0, log=True)
    }
    
    return Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', LogisticRegression(
            solver='liblinear',
            class_weight='balanced',
            **logreg_params
        ))
    ])

def cross_validate_pipeline(pipeline, X, y, n_splits=10, random_seed=42):
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    scores = []
    
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_val)
        scores.append(f1_score(y_val, y_pred, average='weighted'))
    
    return sum(scores) / len(scores)

def optimize_hyperparameters(X_train, y_train, n_trials=4, random_seed=42):
    def objective(trial):
        pipeline = build_model_pipeline(trial)
        return cross_validate_pipeline(pipeline, X_train, y_train, random_seed=random_seed)
    
    study = optuna.create_study(
        direction="maximize",
        sampler=optuna.samplers.TPESampler(seed=random_seed)
    )
    study.optimize(objective, n_trials=n_trials)
    return study.best_params, study.best_value

In [27]:
def train_final_model(params, X_train, y_train):
    vectorizer_type = params['vectorizer']
    # Создаем векторизатор
    vec_params = {k.replace(f"{vectorizer_type}__", ""): v 
                 for k, v in params.items() if k.startswith(f"{vectorizer_type}__")}
    vectorizer = TfidfVectorizer(**vec_params) if vectorizer_type == "tfidf" else CountVectorizer(**vec_params)
    
    # Создаем модель
    logreg_c = params['logreg__C']
    classifier = LogisticRegression(
        solver='liblinear',
        class_weight='balanced',
        C=logreg_c
    )
    
    pipeline = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', classifier)
    ])
    
    return pipeline.fit(X_train, y_train)

def run_experiment(train_data, val_data, test_data, target_col="topic", 
                  n_folds=3, n_trials=4, random_seed=42):
    # Подготовка данных
    X_train_val = pd.concat([train_data['text'], val_data['text']])
    y_train_val = pd.concat([train_data[target_col], val_data[target_col]])
    
    # Оптимизация гиперпараметров
    best_params, best_score = optimize_hyperparameters(
        X_train_val, 
        y_train_val,
        n_trials=n_trials,
        random_seed=random_seed
    )
    print(f"Оптимальные параметры: {best_params}")
    print(f"Лучший F1-score: {best_score:.4f}")
    
    # Обучение финальной модели
    final_model = train_final_model(best_params, train_data['text'], train_data[target_col])
    
    # Оценка на тестовых данных
    y_test = test_data[target_col]
    y_pred = final_model.predict(test_data['text'])
    
    print("\nРезультаты на тестовой выборке:")
    print(classification_report(y_test, y_pred))
    
    return final_model, y_pred

In [28]:
model, predictions = run_experiment(train_df, val_df, test_df, target_col="topic")

[I 2025-03-08 19:26:31,483] A new study created in memory with name: no-name-261c8529-ae95-434d-9d33-08cc4cd094e7
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (1, 1) which is of type tuple.
  warnings.warn(message)
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (1, 2) which is of type tuple.
  warnings.warn(message)
[I 2025-03-08 19:42:39,515] Trial 0 finished with value: 0.8163192431151867 and parameters: {'vectorizer': 'count', 'count__ngram_range': (1, 1), 'count__max_df': 0.7890046601106091, 'count__min_df': 1, 'logreg__C': 0.13066739238053282}. Best is trial 0 with value: 0.8163192431151867.
C:\U

Оптимальные параметры: {'vectorizer': 'count', 'count__ngram_range': (1, 1), 'count__max_df': 0.7890046601106091, 'count__min_df': 1, 'logreg__C': 0.13066739238053282}
Лучший F1-score: 0.8163

Результаты на тестовой выборке:
                   precision    recall  f1-score   support

   69-я параллель       0.67      0.47      0.55        38
           Бизнес       0.54      0.51      0.52       216
      Бывший СССР       0.81      0.87      0.84      1557
              Дом       0.83      0.82      0.83       646
         Из жизни       0.61      0.62      0.61       827
   Интернет и СМИ       0.76      0.73      0.74      1327
             Крым       0.58      0.35      0.44        20
    Культпросвет        0.29      0.20      0.24        10
         Культура       0.86      0.89      0.88      1605
              Мир       0.82      0.82      0.82      4080
  Наука и техника       0.83      0.84      0.84      1579
      Путешествия       0.74      0.70      0.72       192
       

Как видим, качество выросло не сильно, но тем не менее оно и так высоко в особенности по сравнению с бейзлайном
(потсавил кол-во итераций опутны поменьше, тк на моем ноутбуке работае безумно долго)